## 3. Neural Network

#### Importing libraries

In [65]:
import torch
import torch.nn as nn # neural network module
import torch.nn.functional as F # Function 

import torch.optim as optim # Optimizer 

#### Table of content

* [Introduction and random notes](#notes)
    * [Neutral Network module](#nn_module) 
        * Consists of 
            * layers 
            * forward method
    * [Typical training process](#training_process)

* [Define the network](#definition)
* [Create network](#create_network)
* [Run network](#run_network)
    * [Random example](#random_example)
* [Backwards Propagation](#back_prop)
* [Loss Function](#loss)
* [Update weights](#update_weights)

#### Introduction and random notes

Neutral Networds can be constructed using the torch.nn package <a class = "anchor" id="nn_module"></a>
    - Depends on autograd to define models and differentiate them 

An nn.Module contains: 
    - layers 
    - A method: 
        - forward(input) that returns the output

#### Example: MNIST - A network that classifies digit images (Below shows LeNet)

<img src="../images/beginner/mnist.png"></img>

#### A typical training procedure <a class="anchor" id="training_process"></a>

1. Define the neural network that has some learnable parameters (or weights)
2. Iterate over a dataset of inputs
3. Process input through the network 
4. Compute the loss (how far is the output from being correct) 
5. Propagate gradients back into the network's parameters 
6. Update the weights of the network, typically using a simple update rule: 
    - Weight = weight - learning_rate * gradients (Gradient Descent)

## Define the network <a class="anchor" id="definition"></a>

We will be defining a CNN for our network: 

    self.conv1 = nn.Conv2d(1,6,5)

    1 input channel, 6 output channels and 5 * 5 square convolution kernel 

    * 1 --> Only 1 input channel. Ie, greyscale. There can be 3 input channels when we take in RGB. 4 if we do RGBA (Alpha: Transparency). In short, there can be multiple input channels. Not limited to the above

    * 6 -> 6 output channels. 

In [46]:
class Net(nn.Module): # Our definition is the child of the parent class nn.module
    
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(1,6,5) # 1 input image channel, 6 output channels, 5*5 square convolution kernel
        self.conv2 = nn.Conv2d(6,16,5) # 6 input channels
        
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
    
    def forward(self, x): # Does a max pooling over a (2,2) window 
        
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2) # If the size is a square, you can specify a single number 
        x = x.view(-1, self.num_flat_features(x)) # What is num_flat_features
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x 
    
    def num_flat_features(self,x):
        size = x.size()[1:] # Excepted the size to have 3 elements --> first is the number of channels, then width, then height.
        num_features = 1 # Hence, we only look at the number of elements by doing channels * width * height 
        for s in size:
            num_features *= s
        return num_features
        
        

## Creating the network <a class="anchor" id="create_network"></a>

    stride=(1,1) : (x,y). Ie, how much to move in each dimension

In [87]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


## Running the network <a class="anchor" id="run_network"></a>


    We just have to define the forward function, and the backward function (where gradients are computed) is automatically defined for you using autograd.

    We can use any of the Tensor operations in the foward function

    The learnable paramters of a model are returned by: net.parameters()

In [94]:
params = list(net.parameters())
print("Learnable parameters of a model = %d"%len(params))
print()

for i in range(len(params)):
    print(params[i].size())

Learnable parameters of a model = 10

torch.Size([6, 1, 5, 5])
torch.Size([6])
torch.Size([16, 6, 5, 5])
torch.Size([16])
torch.Size([120, 400])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])


#### Trying it with a random input <a class="anchor" id="random_example"></a>

Side note:
    
    torch.nn only supports inputs that are a mini-batch of samples and not a single sample
    Hence, the input torch tensor is 4d --> [nSamples, nChannels, Height, Width]
    

In [95]:
input = torch.randn(1,1,32,32) # 1 batch, 1 image with 32 * 32 
print(input)
print()

out = net(input)
print(out)

tensor([[[[-2.2829,  1.6497, -1.1925,  ..., -0.3277, -0.8164, -0.2679],
          [-0.7825,  0.0575, -0.4530,  ...,  1.2262,  0.5073, -1.2832],
          [-0.9226,  1.9950,  1.2180,  ...,  1.6844,  0.5414, -0.0612],
          ...,
          [ 0.8068, -0.1023,  0.5239,  ..., -1.4450,  0.0538, -0.7288],
          [ 0.5715,  0.6349, -0.6024,  ...,  0.0426,  0.3723, -1.1822],
          [-0.5468,  0.3463,  0.6148,  ...,  1.1758,  0.3162, -1.9371]]]])

tensor([[ 0.0956,  0.1485,  0.0533, -0.0361,  0.0605,  0.0712, -0.0277,
         -0.1027,  0.1241,  0.0266]])


In [96]:
net.zero_grad() # Clear the buffer at each batch 
out.backward(torch.rand(1,10))

## Loss Function <a class="anchor" id="loss"></a>

    A loss function takes the (output, target) pair of inputs, and computes a value that estimates how far away the output is from the target

In [97]:
output = net(input)
print(output)

target = torch.arange(1,11) # ie, range from 0 to 10 
print("Before: ")
print(target)
print()

target = target.view(1, -1) # For it to be [[]] instead of []
print("After: ")
print(target)
print()

criterion = nn.MSELoss() # Setting our criterion to be MSE
loss = criterion(output, target)
print(loss)

tensor([[ 0.0956,  0.1485,  0.0533, -0.0361,  0.0605,  0.0712, -0.0277,
         -0.1027,  0.1241,  0.0266]])
Before: 
tensor([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.])

After: 
tensor([[  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.]])

tensor(38.2061)


## Backprop <a class="anchor" id="back_prop"></a>

If we follow loss in the backward direction, using its .grad_fn attribute, you will see a graph of computations that looks like this: 
    
    input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
      -> view -> linear -> relu -> linear -> relu -> linear
      -> MSELoss
      -> loss

The following illustrates this graph of computations:


In [98]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0]) # Linear function 
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # ReLU

Hence, 
    
    So, when we call loss.backward(), the whole graph is differentiated w.r.t. the loss, and all Tensors in the graph that has requires_grad=True will have their .grad Tensor accumulated with the gradient.
    
    To backpropagate the error all we have to do is to loss.backward(). You need to clear the existing gradients though, else gradients will be accumulated to existing gradients.

    Now we shall call loss.backward(), and have a look at conv1’s bias gradients before and after the backward.
    
    To do so, we can call net.conv1(named by us).bias.grad. Alternatively to look at the gradient of the weights, we can do: 
    net.conv1.weight.grad
    

In [99]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)
print()
print("conv1.weight.grad before backward")
print(net.conv1.weight.grad)
print()

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)
print()
print("conv1.weight.grad before backward")
print(net.conv1.weight.grad)

conv1.bias.grad before backward
tensor([ 0.,  0.,  0.,  0.,  0.,  0.])

conv1.weight.grad before backward
tensor([[[[ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.]]],


        [[[ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.]]],


        [[[ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.]]],


        [[[ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.]]],


        [[[ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  

## Updating the weights <a class="anchor" id="update_weights"></a>

The simplest update rule used in practice is the Stochastic Gradient Descent (SGD):
    
    weight = weight - learning_rate * gradient
    
We can use the torch.optim that implements all these methods

In [101]:
optimizer = optim.SGD(net.parameters(), lr = 0.01) # Learning rate = 0.01

# To update the weights, we can do the following: 
optimizer.zero_grad()
output = net(input)
loss = criterion(output, target) # We have placed the criterion to be MSE Loss 
loss.backward() # To do the backprop, to get the gradient WRT loss 
optimizer.step() # Update the weights